# Code for Raspberry Pi
Here we code everything we need for the experiment

In [121]:
import numpy as np
import matplotlib as plt
import csv
import pandas as pd

Writing a CSV file with csv:

In [122]:
with open('employee_file.csv', mode='w') as employee_file:
    employee_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
    employee_writer.writerow(['John Smith', 'Accounting', 'November'])
    employee_writer.writerow(['Erica Meyers', 'IT', 'March'])


Writing a CSV file from a dictionary with csv:

In [123]:
with open('employee_file2.csv', mode='w') as csv_file:
    fieldnames = ['emp_name', 'dept', 'birth_month']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    writer.writeheader()
    writer.writerow({'emp_name': 'John Smith', 'dept': 'Accounting', 'birth_month': 'November'})
    writer.writerow({'emp_name': 'Erica Meyers', 'dept': 'IT', 'birth_month': 'March'})


Changing an existing CSV file with pandas:

In [124]:
df = pd.read_csv('employee_file2.csv', index_col='Employee', parse_dates=['Hired'], header=0,names=['Employee','Hired','Salary'])
df.to_csv('employee_file2_modified.csv')

Writing a CSV file with pandas:

In [125]:
cities = pd.DataFrame([['Sacramento', 'California'], ['Miami', 'Florida']], columns= ['City','State'])
cities.to_csv('cities.csv', index=False)

In [126]:
df_1 = pd.read_csv('cities.csv')
print(df_1)

         City       State
0  Sacramento  California
1       Miami     Florida


Toy model of CSV for the experiment:

In [127]:
#Take some function f that takes input of raspberry pi and makes it into a list [Timestamp,0 or 1 for each channel,...]
#This is only for storage, not for analysis
count_1 = pd.DataFrame([[15,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0]], columns=['Timestamp', 'Channel 1', 'Channel 2', 'Channel 3', 'Channel 4', 'Channel 5', 'Channel 6', 'Channel 7', 'Channel 8', 'Channel 9', 'Channel 10', 'Channel 11', 'Channel 12', 'Channel 13', 'Channel 14', 'Channel 15', 'Channel 16'])
count_1.to_csv('Toymodel.csv', index=False)

In [128]:
count = pd.read_csv('Toymodel.csv')
print(count)

   Timestamp  Channel 1  Channel 2  Channel 3  Channel 4  Channel 5  \
0         15          0          0          1          0          0   

   Channel 6  Channel 7  Channel 8  Channel 9  Channel 10  Channel 11  \
0          0          0          0          0           1           0   

   Channel 12  Channel 13  Channel 14  Channel 15  Channel 16  
0           0           0           0           0           0  


How to add a row in CSV:

In [129]:
count.loc[len(count)] = [16,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0]
print(count)

   Timestamp  Channel 1  Channel 2  Channel 3  Channel 4  Channel 5  \
0         15          0          0          1          0          0   
1         16          0          0          0          0          0   

   Channel 6  Channel 7  Channel 8  Channel 9  Channel 10  Channel 11  \
0          0          0          0          0           1           0   
1          0          1          0          0           0           1   

   Channel 12  Channel 13  Channel 14  Channel 15  Channel 16  
0           0           0           0           0           0  
1           0           0           0           0           0  


In [133]:
#1)
#Define new function g that takes CSV data and converts into matrix [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]] for data analysis
def g(count):
    event1 = np.zeros((4,4))
    event2 = np.zeros((4,4))

    event_1 = count.to_numpy()[0]
    event_2 = count.to_numpy()[1]
    
    event1[0,:] = event_1[1:5]
    event1[1,:] = event_1[5:9]
    event1[2,:] = event_1[9:13]
    event1[3,:] = event_1[13:17]
    
    event2[0,:] = event_2[1:5]
    event2[1,:] = event_2[5:9]
    event2[2,:] = event_2[9:13]
    event2[3,:] = event_2[13:17]
    #event_1 = count.loc[len(0)]
    print(event1)
    print(event2)
    
    #event_whole = np.array[event_1]
    
g(count)
    

[[0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]]


1) We need a function converting CSV into 4x4 matrix (WDB) (check)
2) We need a function that saves Gyroscope data (CSV) (check)
Data format accelerometer: CSV with quaternions for x,y,z direction
Data format gyroscope: quaternions 
3) We need a function that converts Gyroscope data into rotation matrix (check)
4) We need a function to determine correlation for 4x4 matrix
5) We need a function to filter noise
6) We need a function that converts matrix to vector
7) We need a time comparison function

In [144]:
#3) This function takes the quaternions of the gyroscope and translates them into a 3x3 rotation matrix for spatial vectors
def quaternion_rotation_matrix(Q):
    """
    Covert a quaternion into a full three-dimensional rotation matrix.
 
    Input
    :param Q: A 4 element array representing the quaternion (q0,q1,q2,q3) 
 
    Output
    :return: A 3x3 element matrix representing the full 3D rotation matrix. 
             This rotation matrix converts a point in the local reference 
             frame to a point in the global reference frame.
    """
    # Extract the values from Q
    q0 = Q[0]
    q1 = Q[1]
    q2 = Q[2]
    q3 = Q[3]
     
    # First row of the rotation matrix
    r00 = 2 * (q0 * q0 + q1 * q1) - 1
    r01 = 2 * (q1 * q2 - q0 * q3)
    r02 = 2 * (q1 * q3 + q0 * q2)
     
    # Second row of the rotation matrix
    r10 = 2 * (q1 * q2 + q0 * q3)
    r11 = 2 * (q0 * q0 + q2 * q2) - 1
    r12 = 2 * (q2 * q3 - q0 * q1)
     
    # Third row of the rotation matrix
    r20 = 2 * (q1 * q3 - q0 * q2)
    r21 = 2 * (q2 * q3 + q0 * q1)
    r22 = 2 * (q0 * q0 + q3 * q3) - 1
     
    # 3x3 rotation matrix
    rot_matrix = np.array([[r00, r01, r02],
                           [r10, r11, r12],
                           [r20, r21, r22]])
                            
    return rot_matrix

vec = np.array([0,1,0])
#Mirroring around y and z-axis:
Q = np.array([0,1,0,0])
Q_n = quaternion_rotation_matrix(Q)

vec_n = np.dot(Q_n,vec)
print(Q_n)
print(vec_n)



[[ 1  0  0]
 [ 0 -1  0]
 [ 0  0 -1]]
[ 0 -1  0]


In [167]:
#6) Takes 4x4 matrix M and converts into a spatial vector. Define unit for distance u
def get_vector(M):
    u = 1
    
    row_1_theta = M[0,:]
    row_2_theta = M[2,:]
    
    row_1_phi = M[1,:]
    row_2_phi = M[3,:]
    
    count_1_theta = 1
    count_2_theta = 1
    count_1_phi = 4
    count_2_phi = 4
    
    #gives us which scintillator detected a muon in upper layer (1) for theta
    
    for i in range(3):
        if (row_1_theta[i] != 1):
            count_1_theta = count_1_theta + 1
            continue
        else:
            break
            
    #gives us which scintillator detected a muon in lower layer (3) for theta
    
    for i in range(3):
        if (row_2_theta[i] != 1):
            count_2_theta = count_2_theta + 1
            continue
        else:
            break
    
    #gives us which scintillator detected a muon in upper layer (2) for phi
    
    for i in range(3):
        if (row_1_phi[i] != 1):
            count_1_phi = count_1_phi - 1
            continue
        else:
            break
            
    #gives us which scintillator detected a muon in lower layer (4) for phi
            
    for i in range(3):
        if (row_2_phi[i] != 1):
            count_2_phi = count_2_phi - 1
            continue
        else:
            break
    
    print(count_1_theta)
    print(count_2_theta)
    print(count_1_phi)
    print(count_2_phi)
    
    #we shift our coordinate system to get a centered coordinate system based on unit u
    count_1_theta = count_1_theta - 2.5*u
    count_2_theta = count_2_theta - 2.5*u
    count_1_phi = count_1_phi - 2.5*u
    count_2_phi = count_2_phi - 2.5*u

    print(count_1_theta)
    print(count_2_theta)
    print(count_1_phi)
    print(count_2_phi)
    
    delta_y = count_2_theta-count_1_theta
    delta_x = count_2_phi-count_1_phi
    delta_z = -2*u
    
    #Get the angles corresponding to the triangle constructed above
    #theta = np.arctan(2/(count_1_theta-count_2_theta))
    #phi = np.arctan(2/(count_1_phi-count_2_phi))
    
    ##################################CORRECTION IDEA#####################################
    
    if delta_y == 0:
        delta_y = 1
    
    print("Delta_y:")
    print(delta_y)
    print("Delta_x:")
    print(delta_x)
    
    vector_alt = np.array([delta_x, delta_y, delta_z])
        
    print("Alternative vector:")
    print(vector_alt)
    
    r = np.sqrt(delta_x**2+delta_y**2+delta_z**2)
    
    theta_alt = np.arccos((-delta_z)/r)
    print("Alternative angle:")
    print(theta_alt)
    
    ######################################################################################
    
    #print("Calculated angle theta:")
    #print(theta)
    #print("Calculated angle phi:")
    #print(phi)
    
    #Get the vector by transforming back into cartesian coordinates
    #x = np.sin(theta)*np.cos(phi)
    #y = np.sin(theta)*np.sin(phi)
    #z = np.cos(theta)
    
    #vector = np.array([x,y,z])
    #print("Calculated vector:")
    #print(vector)
    
    #rotate vector with rotation matrix
    Q = np.array([0,1,0,0])
    Q_n = quaternion_rotation_matrix(Q)
    
    vector_n = np.dot(Q_n,vector_alt)
    print("Rotated vector:")
    print(vector_n)
    
    #return(theta, phi, vector_n)


M = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,1,0,0]])
get_vector(M)

    
    

1
3
2
4


In [ ]:
Test
